In [2]:
import json
import math
import pytz
from datetime import datetime,timedelta,timezone
from dateutil.tz import tzutc
from dateutil.relativedelta import relativedelta
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math
import csv
from scipy.interpolate import interp1d
from scipy.stats import pearsonr, mode
from scipy.signal import savgol_filter
import xgboost as xgb
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE

addDataPrefix='/Users/sorush/My Drive/Documents/Educational/TAMU/Research/Trial/Data/11-5-21-11-15-21'
if not os.path.exists(addDataPrefix):
    addDataPrefix='/home/grads/s/sorush.omidvar/CGMDataset/Trial/Data/11-5-21-11-15-21'
addUserInput=os.path.join(addDataPrefix,'User inputted')
addHKCM=os.path.join(addDataPrefix,'hk+cm')
addCGM=os.path.join(addDataPrefix,'CGM')
addE4=os.path.join(addDataPrefix,'E4')

exempts=['p2']

pd.options.mode.chained_assignment = None  # default='warn'
plt.style.use({'figure.facecolor':'white'})

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 50)

In [3]:
# if os.path.exists(os.path.join(addDataPrefix,'All_meals.pkl')):
#     os.remove(os.path.join(addDataPrefix,'All_meals.pkl'))
os.chdir(addUserInput)
if not os.path.exists(os.path.join(addDataPrefix,'All_meals.pkl')):    
    df=[]
    for root, dirs, files in os.walk(addUserInput):
        for file in files:
            if '.csv' in file.lower():
                if 'meals' in file.lower():
                    participantName=file[:file.find('Meals')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file)
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp.rename(columns={'startTime':'StartTime'}, inplace=True)
                    dfTemp['StartTime']=pd.to_datetime(dfTemp['StartTime'])
                    dfTemp['FinishTime']=pd.to_datetime(dfTemp['FinishTime'])

                    dfTemp['StartTime']-=pd.DateOffset(hours=5)#fixing the time zone issue
                    dfTemp['FinishTime']-=pd.DateOffset(hours=5)#fixing the time zone issue            
                    dfTemp.sort_values(["Participant",'StartTime'],ascending = (True, True),inplace=True)
                    if len(dfTemp.columns)!=10:
                        print("MAYDAY. Error in reading csv")
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    print("reading is done")
    dfMeal=df    
    dfMeal.to_pickle(os.path.join(addDataPrefix,'All_meals.pkl')) 
else:
    dfMeal=pd.read_pickle(os.path.join(addDataPrefix,'All_meals.pkl'))


# if os.path.exists(os.path.join(addDataPrefix,'All_cgm.pkl')):
#     os.remove(os.path.join(addDataPrefix,'All_cgm.pkl'))
if not os.path.exists(os.path.join(addDataPrefix,'All_cgm.pkl')):
    os.chdir(addCGM)
    df=[]
    for root, dirs, files in os.walk(addCGM):
        for file in files:
            if '.csv' in file.lower():
                if 'processed' in file.lower():
                    participantName=file[:file.find('Processed')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp = pd.read_csv(file)
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp['Time']=pd.to_datetime(dfTemp['Time'])
                    dfTemp.sort_values(["Participant","Time"],ascending = (True,True),inplace=True)
                    if len(dfTemp.columns)!=4:
                        print("MAYDAY. Error in reading csv")
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    print("reading is done")
    dfCGM=df
    dfCGM['Abbot'].interpolate(method='linear',limit_direction='both',axis=0,inplace=True)
    dfCGM['Dexcom'].interpolate(method='linear',limit_direction='both',axis=0,inplace=True)
    dfCGM.to_pickle(os.path.join(addDataPrefix,'All_cgm.pkl')) 
else:
    dfCGM=pd.read_pickle(os.path.join(addDataPrefix,'All_cgm.pkl'))



    
# if os.path.exists(os.path.join(addDataPrefix,'All_cm.pkl')):
#     os.remove(os.path.join(addDataPrefix,'All_cm.pkl'))
os.chdir(addHKCM)
if not os.path.exists(os.path.join(addDataPrefix,'All_cm.pkl')):
    df=[]
    for root, dirs, files in os.walk(addHKCM):
        for file in files:
            if '.csv' in file.lower():
                if 'cm' in file.lower() and 'modified' in file.lower():
                    participantName=file[:file.find('_cm')]
                    if participantName in exempts:
                        print("Exemption...",file)
                        continue
                    print("Reading ...",file)
                    dfTemp=pd.read_csv(file)
                    print("File is read")
                    dfTemp.insert(0,'Participant',participantName)
                    dfTemp.rename(columns={'Date':'Time'}, inplace=True)
                    dfTemp['Time']=pd.to_datetime(dfTemp['Time'])
                    dfTemp.insert(len(dfTemp.columns),'|Ax|+|Ay|+|Az|',dfTemp['Ax'].abs()+dfTemp['Ay'].abs()+dfTemp['Az'].abs()+0.001)#this is to avoid 0 later on for feature calculation
                    dfTemp.insert(len(dfTemp.columns),'|Yaw|+|Roll|+|Pitch|',dfTemp['Yaw'].abs()+dfTemp['Roll'].abs()+dfTemp['Pitch'].abs())
                    dfTemp.insert(len(dfTemp.columns),'RotationalToLinear',dfTemp['|Yaw|+|Roll|+|Pitch|']/dfTemp['|Ax|+|Ay|+|Az|'])
                    print("modified")
                    dfTemp.sort_values(['Time'],ascending = (True),inplace=True)
                    print("sorted")
                    if len(dfTemp.columns)!=14:
                        print("MAYDAY. Error in reading csv")
                        print(dfTemp.columns)
                        break
                    if len(df)!=0:
                        frames=[dfTemp,df]
                        df=pd.concat(frames)
                    else:
                        df=dfTemp
    dfCM=df
    print("Processing is done")
    dfCM.to_pickle(os.path.join(addDataPrefix,'All_cm.pkl')) 
else:
    dfCM = pd.read_pickle(os.path.join(addDataPrefix,'All_cm.pkl'))

Exemption... p2Meals.csv
Reading ... p4Meals.csv
Reading ... p3Meals.csv
Reading ... p1Meals.csv
reading is done
Reading ... p1Processed.csv
Reading ... p4Processed.csv
Reading ... p3Processed.csv
Exemption... p2Processed.csv
reading is done
Reading ... p4_cm_all_modified.csv
File is read
modified
sorted
Reading ... p1_cm_all_modified.csv
File is read
modified
sorted
Exemption... p2_cm_all_modified.csv
Reading ... p3_cm_all_modified.csv
File is read
modified
sorted
Processing is done


In [4]:
def featureExtractorMotion(df):
    f1=df['RotationalToLinear'].mean()
    f2=df['|Ax|+|Ay|+|Az|'].mean()
    f5=df['|Yaw|+|Roll|+|Pitch|'].mean()
    return [f1,f2,f5]

def featureExtractorCGM(df):
    mean=df['Abbot'].mean()
    std=df['Abbot'].std()
    max=df['Abbot'].max()
    min=df['Abbot'].min()
    return mean, std, min,max

MINIMUM_POINT_CM=10
WINDOW_LENGTH=timedelta(minutes=2)
# WINDOW_STEP=timedelta(minutes=2)
MEAL_PORTION_RELAXATION=timedelta(seconds=10)
START_OF_TRIAL = datetime.strptime('11 06 2021-02:00:00', '%m %d %Y-%H:%M:%S')#to handle the daylight saving issue in apple watches
END_OF_TRIAL = datetime.strptime('11 07 2021-00:00:00', '%m %d %Y-%H:%M:%S')

dfCM=dfCM[dfCM['Time']>=START_OF_TRIAL]
dfCM=dfCM[dfCM['Time']<END_OF_TRIAL]

dfMeal=dfMeal[dfMeal['Time']>=START_OF_TRIAL]
dfMeal=dfMeal[dfMeal['Time']<END_OF_TRIAL]

participants=dfCM['Participant'].to_list()
participants=list(set(participants))

windowStart=START_OF_TRIAL
windowEnd=windowStart+WINDOW_LENGTH
for participant in participants:
    participantDataList=[]
    print("Participant ",participant," is started")
    for counter in tqdm(range(int((END_OF_TRIAL-START_OF_TRIAL).seconds/WINDOW_LENGTH.seconds))):
        dfTempCM=dfCM[(dfCM['Time']>=windowStart) & (dfCM['Time']<=windowEnd) & (dfCM['Participant']==participant)]
        dfTempMeal=dfMeal[(dfMeal['StartTime']-MEAL_PORTION_RELAXATION<=windowStart) & (dfMeal['FinishTime']+MEAL_PORTION_RELAXATION>=windowEnd)& (dfMeal['Participant']==participant)]
        if(len(dfTempCM)<MINIMUM_POINT_CM):
            windowStart+=WINDOW_LENGTH
            windowEnd+=WINDOW_LENGTH
            continue
        tempList=featureExtractorMotion(dfTempCM)
        if(len(dfTempMeal)!=0):
            tempList.append(1)
        else:
            tempList.append(0)
        participantDataList.append(tempList)
        windowStart+=WINDOW_LENGTH
        windowEnd+=WINDOW_LENGTH
participantDataArray=np.asarray(participantDataList)

In [95]:
def STMI_XGBoost(xTrain,xVal,xTest,yTrain,yVal,yTest):
    accuracyBest=0
    for maxDepth in tqdm(np.arange(3,30)):
        for estimator in np.arange(5,50,2):
            for threshold in np.arange(0.3,0.7,0.1):
                clf = xgb.XGBClassifier(n_jobs=16,n_estimators=estimator,max_depth=maxDepth,objective = "binary:logistic",eval_metric = "logloss",use_label_encoder =False)
                clf.fit(xTrain,yTrain)
                predictionsVal = clf.predict_proba(xVal)
                predictionsVal=predictionsVal[:,1]
                predictionsVal[predictionsVal>=threshold]=1
                predictionsVal[predictionsVal<threshold]=0

                confMatrix=sklearn.metrics.confusion_matrix(yVal,predictionsVal)
                accuracy=sklearn.metrics.accuracy_score(yVal,predictionsVal)
                recall=sklearn.metrics.recall_score(yVal,predictionsVal)
                precision=sklearn.metrics.precision_score(yVal,predictionsVal)

                if accuracy>accuracyBest:
                    confMatrixBest=confMatrix
                    accuracyBest=accuracy
                    modelBest=clf
                    recallBest=recall
                    precisionBest=precision
                    thresholdBest=threshold
    print("***********Val:")
    print(confMatrixBest)
    print("Accuracy:",np.round(100*accuracyBest,0),"Recall:",np.round(100*recallBest,0),"Precision:",np.round(100*precisionBest,0))
    print("***********Test:")
    predictionsTest=modelBest.predict_proba(xTest)
    predictionsTest=predictionsTest[:,1]
    predictionsTest[predictionsTest>=thresholdBest]=1
    predictionsTest[predictionsTest<thresholdBest]=0    
    
    confMatrix=sklearn.metrics.confusion_matrix(yTest,predictionsTest)
    accuracy=sklearn.metrics.accuracy_score(yTest,predictionsTest)
    recall=sklearn.metrics.recall_score(yTest,predictionsTest)
    precision=sklearn.metrics.precision_score(yTest,predictionsTest)
    
    print(confMatrixBest)
    print("Accuracy:",np.round(100*accuracy,0),"Recall:",np.round(100*recall,0),"Precision:",np.round(100*precision,0))

def dataBalancer(xTrain,xVal,yTrain,yVal):
    oversample = SMOTE()
    xVal, yVal = oversample.fit_resample(xVal, yVal)
    xTrain, yTrain = oversample.fit_resample(xTrain, yTrain)

    return xTrain,xVal,yTrain,yVal

def testTrainSplitFunc(data,randomSeed):
    random.seed(randomSeed)
    np.random.shuffle(data)

    dataX=data[:,0:3]
    dataY=data[:,3]
    dataXNorm=dataX-dataX.mean(axis=0)
    dataXNorm/=dataXNorm.std(axis=0)

    stratidiedSampling = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=randomSeed)
    for trainIndex, testIndex in stratidiedSampling.split(dataX, dataY):
        xTrain,xTest=dataX[trainIndex],dataX[testIndex]
        yTrain,yTest=dataY[trainIndex],dataY[testIndex]

    stratidiedSampling = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=randomSeed)
    for trainIndex, valIndex in stratidiedSampling.split(xTrain, yTrain):
        xTrain,xVal=dataX[trainIndex],dataX[valIndex]
        yTrain,yVal=dataY[trainIndex],dataY[valIndex]

    return xTrain,xVal,xTest,yTrain,yVal,yTest

randomSeed=random.randrange(100)
xTrain,xVal,xTest,yTrain,yVal,yTest=testTrainSplitFunc(participantDataArray,randomSeed=randomSeed)
xTrain,xVal,yTrain,yVal=dataBalancer(xTrain,xVal,yTrain,yVal)
print(yTest.mean())
print(yTrain.mean())
STMI_XGBoost(xTrain,xVal,xTest,yTrain,yVal,yTest)

0.15228426395939088
0.5


100%|██████████| 27/27 [00:27<00:00,  1.03s/it]

***********Val:
[[109  11]
 [ 52  68]]
Accuracy: 74.0 Recall: 57.0 Precision: 86.0
***********Test:
[[109  11]
 [ 52  68]]
Accuracy: 89.0 Recall: 73.0 Precision: 61.0


In [46]:
def STMI_XGBoost(trainData,testData,trainLabels,testLabels):
    mseBest=math.inf
    depthBest=float('nan')
    estimatorBest=float('nan')
    modelBest=''

    for numEstimator in np.arange(10,120,10):
        for maxDepth in np.arange(3,10):
            clf = xgb.XGBRegressor(n_estimators =numEstimator,max_depth=maxDepth,objective ='reg:squarederror',eval_metric = "logloss")
            clf.fit(trainData, trainLabels)
            testPrediction = clf.predict(testData)
            mre=[]
            for counter in range(0,len(testPrediction)):
                mre.append(math.fabs(testPrediction[counter]-testLabels[counter])/testLabels[counter])
            mre=np.asarray(mre)
            mre=np.mean(mre)
            if mre<mseBest:
                mreBest=mre
                depthBest=maxDepth
                estimatorBest=numEstimator
                modelBest=clf
                predictionBest=testPrediction
    print(clf.feature_importances_)
    return modelBest,mreBest,predictionBest,depthBest,estimatorBest

def aucCalculator(timeTemp,dataTemp,kernelNumber):
    kernelMeans=np.linspace(0,24,kernelNumber+2)
    kernelMeans=np.delete(kernelMeans,0)
    kernelMeans=np.delete(kernelMeans,-1)
    kernelMeans=kernelMeans*3600

    kernelSTD=2*3600
    aucData=[]
    for kernelMean in kernelMeans:
        kernelTemp=np.exp(-((timeTemp-kernelMean)**2)/(2*kernelSTD**2))
        tempAuc=kernelTemp*dataTemp
        tempAuc=np.trapz(tempAuc,x=timeTemp)
        aucData.append(tempAuc/3600)

    return aucData

def statisticalFeature(timeTemp,dataTemp):
    featureData=[]
    featureList=[]

    featureData.append(np.mean(dataTemp))
    featureData.append(np.max(dataTemp)-np.min(dataTemp))
    featureData.append(np.var(dataTemp))

    featureData.append(np.mean(np.diff(dataTemp,n=1)))
    # featureData.append(np.median(np.diff(dataTemp,n=1)))
    featureData.append(np.max(np.diff(dataTemp,n=1)))
    featureData.append(np.min(np.diff(dataTemp,n=1)))
    # featureData.append(np.var(np.diff(dataTemp,n=1)))

    featureData.append(np.mean(np.diff(dataTemp,n=2)))
    # featureData.append(np.median(np.diff(dataTemp,n=2)))
    # featureData.append(np.max(np.diff(dataTemp,n=2)))
    # featureData.append(np.min(np.diff(dataTemp,n=2)))
    # featureData.append(np.var(np.diff(dataTemp,n=2)))

    featureData.append(np.trapz(dataTemp)/3600)
    return featureData

def featureExtractor(df,contextFlag):
    featureList=[]
    sensors=['CGM','EDA', 'Acc','Temp', 'HR','Cal']


    #-----------------I removed the STEP as it was almost always zero. We should look into it later------------
    featureLabels=[]
    for sensor in sensors:

        varData=df[df['Parameter']==sensor]
        varTime=varData['Time'].tolist()
        varData=varData['Value'].tolist()

        varTime=np.asarray(varTime)
        varData=np.asarray(varData)
        if contextFlag:
            featureList.extend(statisticalFeature(varTime,varData))
            featureLabels.append([sensor,'mean'])
            featureLabels.append([sensor,'max'])
            featureLabels.append([sensor,'var'])
            featureLabels.append([sensor,'S-mean'])
            featureLabels.append([sensor,'S-max'])
            featureLabels.append([sensor,'S-min'])
            featureLabels.append([sensor,'SS-mean'])
            featureLabels.append([sensor,'trapz'])
        if sensor=='CGM':
            featureList.extend(aucCalculator(varTime,varData,9))
            for i in range(0,9):
                featureLabels.append([sensor,'auc'+str(i)])
            featureList.extend(aucCalculator(varTime,varData,6))
            for i in range(0,6):
                featureLabels.append([sensor,'auc'+str(i)])
    print("Used features",featureLabels)
    #REDO IT WITH FITBIT AND E4
    #FOR MEAL TO MEAL PERSPECTIVE, MAKE THE WHOLE DAY INTO 3 CHUNCKS (MEAL TO MEAL)
    #PLOT DAY-DAY VS MEAL TO MEAL AND SHOW THEM HOW WE ARE SEAPRATING THINGS (CGM,EDA,...)

    #https://github.com/fraunhoferportugal/tsfel/tree/9319db4368303cf10adb3aeb72cd4235a8085307
    return featureList

def testTrainSplitFuncAux(dfSensorTemp,dfMacroTemp,contextFlag):
    sensorDataList=[]
    macroDataList=[]
    days=list(set(dfSensorTemp['Date']))

    for day in days:
        dfSensorTempDay=dfSensorTemp[dfSensorTemp['Date']==day]
        sensorDataList.append(featureExtractor(dfSensorTempDay,contextFlag))
        dfMacroTempDay=dfMacroTemp[dfMacroTemp['Date']==day]
        totalCarb=dfMacroTempDay['Carb'].sum()
        totalFat=dfMacroTempDay['Fat'].sum()
        totalProtein=dfMacroTempDay['Protein'].sum()
        macroDataList.append([totalCarb,totalFat,totalProtein])

    return sensorDataList,macroDataList

def featureNormalizer(trainData,testData):
    # repeatingNum=trainData.shape
    # repeatingNum=repeatingNum[0]
    #
    # columnMeans=np.mean(trainData,axis=0)
    # columnMeans = np.repeat(columnMeans[:, np.newaxis], repeatingNum, axis=1)
    # columnMeans=np.transpose(columnMeans)
    # trainData-=columnMeans
    #
    # columnSTD=np.std(trainData,axis=0)
    # columnSTD = np.repeat(columnSTD[:, np.newaxis], repeatingNum, axis=1)
    # columnSTD=np.transpose(columnSTD)
    # trainData/=columnSTD
    #
    # repeatingNum=testData.shape
    # repeatingNum=repeatingNum[0]
    #
    # columnMeans=np.mean(trainData,axis=0)
    # columnMeans = np.repeat(columnMeans[:, np.newaxis], repeatingNum, axis=1)
    # columnMeans=np.transpose(columnMeans)
    # testData-=columnMeans
    #
    # columnSTD=np.std(trainData,axis=0)
    # columnSTD = np.repeat(columnSTD[:, np.newaxis], repeatingNum, axis=1)
    # columnSTD=np.transpose(columnSTD)
    # testData/=columnSTD
    scaler = StandardScaler()
    trainData = scaler.fit_transform(trainData)
    testData = scaler.fit_transform(testData)

    return trainData,testData

def testTrainSplitFunc(dfSensor, dfMacro,randomSeed,contextFlag):
    days=list(set(dfSensor['Date']))
    random.seed(randomSeed)
    random.shuffle(days)
    daysTrain=days[0:6]
    daysTest=days[6:9]

    dfSensorTrain = dfSensor[dfSensor['Date'].isin(daysTrain)]
    dfSensorTest = dfSensor[dfSensor['Date'].isin(daysTest)]

    dfMacroTrain= dfMacro[dfMacro['Date'].isin(daysTrain)]
    dfMacroTest= dfMacro[dfMacro['Date'].isin(daysTest)]

    trainData,trainLabel=testTrainSplitFuncAux(dfSensorTrain,dfMacroTrain,contextFlag)
    testData,testLabel=testTrainSplitFuncAux(dfSensorTest,dfMacroTest,contextFlag)

    trainData=np.asarray(trainData)
    testData=np.asarray(testData)

    trainLabel=np.asarray(trainLabel)
    testLabel=np.asarray(testLabel)

    trainLabel=trainLabel[:,0]
    testLabel=testLabel[:,0]
    trainData,testData=featureNormalizer(trainData,testData)
    # for i in range(0,testData.shape[1]):
    #     plt.figure()
    #     plt.plot(testData[:,i])
    #     plt.title(str(i))
    return trainData,testData,trainLabel,testLabel

randomSeed=random.randrange(100)
trainData,testData,trainLabel,testLabel=testTrainSplitFunc(dfInterp,dfMacro,randomSeed=randomSeed,contextFlag=True)
_,mreCGMCon,predictionCGMCon,_,_=STMI_XGBoost(trainData,testData,trainLabel,testLabel)
pearsonCGMCon=pearsonr(predictionCGMCon,testLabel)
pearsonCGMCon=pearsonCGMCon[0]

trainData,testData,trainLabel,testLabel=testTrainSplitFunc(dfInterp,dfMacro,randomSeed=randomSeed,contextFlag=False)
_,mreCGM,predictionCGM,_,_=STMI_XGBoost(trainData,testData,trainLabel,testLabel)
pearsonCGM=pearsonr(predictionCGM,testLabel)
pearsonCGM=pearsonCGM[0]

plt.scatter(x=testLabel,y=predictionCGM,color='red',label='CGM')
plt.scatter(x=testLabel,y=predictionCGMCon,color='blue',label='CGM+Context')
plt.plot([200,600],[200,600],':k')
# plt.scatter(x=daysTest,y=predictionBestCGMContext,label='Prediction CGM+Con')
# plt.scatter(x=daysTest,y=predictionBestCGM,label='Prediction CGM')
plt.annotate('CGM Error'+str(round(100*mreCGM,2))+'%', xy=(1.05, 0.95), xycoords='axes fraction')
plt.annotate('CGMCon Error'+str(round(100*mreCGMCon,2))+'%', xy=(1.05, 0.85), xycoords='axes fraction')

plt.annotate('CGM Pearson'+str(round(100*pearsonCGM,2))+'%', xy=(1.05, 0.75), xycoords='axes fraction')
plt.annotate('CGMCon Pearson'+str(round(100*pearsonCGMCon,2))+'%', xy=(1.05, 0.65), xycoords='axes fraction')

plt.xlabel('Actual Carb [gr]')
plt.ylabel('Predicted Carb [gr]')
plt.legend(loc='upper left')
plt.savefig('C:\GitHub\Carb.png')

NameError: name 'dfInterp' is not defined